In [2]:
!pip install faiss-cpu numpy==1.26.3 
!pip install openai==1.3.3 scikit-learn 
!pip install sentence-transformers==2.2.2 
!pip install tenacity==8.2.3 tiktoken==0.5.1 
!pip install torch transformers==4.38.1 umap-learn==0.5.5 urllib3==1.26.6 

Found existing installation: umap 0.1.1
Uninstalling umap-0.1.1:
  Successfully uninstalled umap-0.1.1


In [3]:
!pip install pypdf

In [1]:
!rm -rf ~/.cache/*
!rm -rf /tmp/*

In [2]:
import torch
from raptor import BaseSummarizationModel, BaseQAModel, BaseEmbeddingModel, RetrievalAugmentationConfig
from transformers import AutoTokenizer, pipeline

In [3]:
# if you want to use the Gemma, you will need to authenticate with HuggingFace, Skip this step, if you have the model already downloaded
from huggingface_hub import login
login()

In [4]:
from transformers import AutoTokenizer, pipeline
import torch

# You can define your own Summarization model by extending the base Summarization Class. 
class GEMMASummarizationModel(BaseSummarizationModel):
    def __init__(self, model_name="google/gemma-2b-it"):
        # Initialize the tokenizer and the pipeline for the GEMMA model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.summarization_pipeline = pipeline(
            "text-generation",
            model=model_name,
            model_kwargs={"torch_dtype": torch.bfloat16},
            device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),  # Use "cpu" if CUDA is not available
        )

    def summarize(self, context, max_tokens=150):
        # Format the prompt for summarization
        messages=[
            {"role": "user", "content": f"Write a summary of the following, including as many key details as possible: {context}:"}
        ]
        
        prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # Generate the summary using the pipeline
        outputs = self.summarization_pipeline(
            prompt,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )
        
        # Extracting and returning the generated summary
        summary = outputs[0]["generated_text"].strip()
        return summary

In [5]:
class GEMMAQAModel(BaseQAModel):
    def __init__(self, model_name= "google/gemma-2b-it"):
        # Initialize the tokenizer and the pipeline for the model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.qa_pipeline = pipeline(
            "text-generation",
            model=model_name,
            model_kwargs={"torch_dtype": torch.bfloat16},
            device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
        )

    def answer_question(self, context, question):
        # Apply the chat template for the context and question
        messages=[
              {"role": "user", "content": f"Given Context: {context} Give the best full answer amongst the option to question {question}"}
        ]
        prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        # Generate the answer using the pipeline
        outputs = self.qa_pipeline(
            prompt,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )
        
        # Extracting and returning the generated answer
        answer = outputs[0]["generated_text"][len(prompt):]
        return answer

In [6]:
from sentence_transformers import SentenceTransformer
class SBertEmbeddingModel(BaseEmbeddingModel):
    def __init__(self, model_name="sentence-transformers/multi-qa-mpnet-base-cos-v1"):
        self.model = SentenceTransformer(model_name)

    def create_embedding(self, text):
        return self.model.encode(text)

In [7]:
RAC = RetrievalAugmentationConfig(summarization_model=GEMMASummarizationModel(), qa_model=GEMMAQAModel(), embedding_model=SBertEmbeddingModel())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-06-05 13:00:25,455 - Use pytorch device_name: cuda
2024-06-05 13:00:25,455 - Load pretrained SentenceTransformer: sentence-transformers/multi-qa-mpnet-base-cos-v1


In [8]:
from raptor import RetrievalAugmentation 

In [9]:
RA = RetrievalAugmentation(config=RAC)

2024-06-05 12:06:49,515 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <__main__.GEMMASummarizationModel object at 0x7f9677f31b20>
            Embedding Models: {'EMB': <__main__.SBertEmbeddingModel object at 0x7f9677eeee20>}
            Cluster Embedding Model: EMB
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2024-06-05 12:06:49,516 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summariz

In [10]:
from pypdf import PdfReader
FILE_PATH = './RaptorResearchPaper.pdf'
reader = PdfReader(FILE_PATH)
num_of_pages = len(reader.pages)
print(f"Number of Pages: {num_of_pages}")

Number of Pages: 23


In [11]:
text = ""
for page_num in range(3, 13):
  page = reader.pages[page_num]
  text += page.extract_text()+" "

In [12]:
print(text)

Published as a conference paper at ICLR 2024
Given a set of Ntext segments, each represented as a d-dimensional dense vector embedding, the
likelihood of a text vector, x, given its membership in the kthGaussian distribution, is denoted by
P(x|k) =N(x;µk,Σk). The overall probability distribution is a weighted combination P(x) =PK
k=1πkN(x;µk,Σk), where πksignifies the mixture weight for the kthGaussian distribution.
The high dimensionality of vector embeddings presents a challenge for traditional GMMs, as dis-
tance metrics may behave poorly when used to measure similarity in high-dimensional spaces (Ag-
garwal et al., 2001). To mitigate this, we employ Uniform Manifold Approximation and Projection
(UMAP), a manifold learning technique for dimensionality reduction (McInnes et al., 2018). The
number of nearest neighbors parameter, nneighbors , in UMAP determines the balance between
the preservation of local and global structures. Our algorithm varies nneighbors to create a hierar-
chica

In [14]:
RA.add_documents(text)

2024-06-05 12:09:30,614 - Creating Leaf Nodes


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

2024-06-05 12:11:42,932 - Node Texts Length: 2377, Summarized Text Length: 2512


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-06-05 12:11:51,361 - Node Texts Length: 2258, Summarized Text Length: 2390


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-06-05 12:11:51,416 - Constructing Layer 2
2024-06-05 12:11:51,417 - Stopping Layer construction: Cannot Create More Layers. Total Layers in tree: 2
2024-06-05 12:11:51,418 - Successfully initialized TreeRetriever with Config 
        TreeRetrieverConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Context Embedding Model: EMB
            Embedding Model: <__main__.SBertEmbeddingModel object at 0x7f9677eeee20>
            Num Layers: None
            Start Layer: None
        


In [15]:
print("Completed")

Completed


In [16]:
question = "What is RAPTOR?"
answer = RA.answer_question(question = question)
print("Answer : ",answer)

2024-06-05 12:11:51,429 - Using collapsed_tree


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer :  Sure, here's a summary of the text:

**RAPTOR is a novel tree-based retrieval system that augments the parametric knowledge of large language models with contextual information at various levels of abstraction.**

**Key features of RAPTOR:**

- **Tree traversal:** Selects the top-k most relevant root nodes based on cosine similarity to the query embedding.
- **Collapsed tree:** Uses a hierarchical structure to capture information from different tree layers.

**Advantages of RAPTOR:**

- More comprehensive information retrieval due to intermediate layers' clustering.
- Effective retrieval of both general themes and specific details.

**Performance:**

- Outperforms traditional retrieval methods.
- Sets new performance benchmarks on several question-answering tasks.


In [19]:
SAVE_PATH = 'RAPTOR_index'
RA.save(SAVE_PATH)

2024-06-05 12:14:08,138 - Tree successfully saved to RAPTOR_index


In [8]:
from raptor import RetrievalAugmentation
SAVE_PATH = 'RAPTOR_index'
RA = RetrievalAugmentation(config = RAC ,tree=SAVE_PATH)

2024-06-05 13:00:35,814 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <__main__.GEMMASummarizationModel object at 0x7f2f12a42370>
            Embedding Models: {'EMB': <__main__.SBertEmbeddingModel object at 0x7f2f0c6777c0>}
            Cluster Embedding Model: EMB
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2024-06-05 13:00:35,815 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summariz

In [9]:
question = "What is RAPTOR?"
answer = RA.answer_question(question = question)
print("Answer : ",answer)

2024-06-05 13:00:36,375 - Using collapsed_tree


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer :  Sure, here's a summary of the provided context about RAPTOR:

**RAPTOR** is a novel tree-based retrieval system that enhances the parametric knowledge of large language models with contextual information at various levels of abstraction.

**Key features of RAPTOR:**

- It employs recursive clustering and summarization techniques to create a hierarchical tree structure that captures information from different tree layers.
- This hierarchical structure allows RAPTOR to leverage both general themes and specific details of the query.
- It provides both the original text and higher-level summaries for retrieval, enabling more comprehensive information access.
- RAPTOR outperforms traditional retrieval methods on various question-answering tasks, including the QASPER dataset.


In [10]:
tree = RA.tree

In [11]:
nodes = tree.all_nodes
nodes

{2: <raptor.tree_structures.Node at 0x7f2f151c29a0>,
 1: <raptor.tree_structures.Node at 0x7f2f12bd5610>,
 3: <raptor.tree_structures.Node at 0x7f2f12a42400>,
 6: <raptor.tree_structures.Node at 0x7f3010072a90>,
 5: <raptor.tree_structures.Node at 0x7f2f08052f10>,
 0: <raptor.tree_structures.Node at 0x7f2f0805e700>,
 7: <raptor.tree_structures.Node at 0x7f2f0805e430>,
 4: <raptor.tree_structures.Node at 0x7f2f0805e4f0>,
 8: <raptor.tree_structures.Node at 0x7f2f0805e730>,
 12: <raptor.tree_structures.Node at 0x7f2f0805e670>,
 9: <raptor.tree_structures.Node at 0x7f2f0805e8b0>,
 10: <raptor.tree_structures.Node at 0x7f2f0805e7f0>,
 11: <raptor.tree_structures.Node at 0x7f2f0805e2e0>,
 13: <raptor.tree_structures.Node at 0x7f2f0805ea30>,
 14: <raptor.tree_structures.Node at 0x7f2f0805e940>,
 15: <raptor.tree_structures.Node at 0x7f2f0805e130>,
 16: <raptor.tree_structures.Node at 0x7f2f0805eb80>,
 19: <raptor.tree_structures.Node at 0x7f2f0805e280>,
 17: <raptor.tree_structures.Node at 0

In [12]:
len(nodes)

126

In [13]:
def print_tree_layers(root_nodes):
    all_nodes = tree.all_nodes
    current_layer = list(root_nodes.values())
    level = 0
    while current_layer:
        print(f"============================================level {level}============================================")
        next_layer = []
        for node in current_layer:
            print(f"Index: {node.index}, Text: {node.text}\n")
            next_layer.extend(all_nodes.get(child_index) for child_index in node.children)
            
        current_layer = next_layer
        level+=1

In [14]:
print_tree_layers(tree.root_nodes)

============================================level 0============================================
Index: 119, Text: <bos><start_of_turn>user
Write a summary of the following, including as many key details as possible: <bos><start_of_turn>user Write a summary of the following, including as many key details as possible:  As shown in Table 3, RAPTOR consistently outperforms BM25 and DPR across all three Language Models on the QASPER dataset  RAPTOR’s F-1 Match scores are 53 1%, 55 7%, and 36 6% when using GPT-3, GPT-4, and UnifiedQA, respectively  These scores surpass DPR by margins of 1 8, 2 7, and 4  Abstract” reflects performance when only the title and abstract of the papers are used for context RAPTOR outperforms the established baselines BM25 and DPR across all tested language models Specifically, RAPTOR’s F-1 scores are at least 1 8% points higher than DPR and at least 5 3% points higher than BM25 Retriever GPT-3 F-1 Match GPT-4 F-1 Match UnifiedQA F-1 Match   RAPTOR outperforms the 

In [15]:
!conda install python-graphviz -y 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/studio-lab-user/.conda/envs/default

  added / updated specs:
    - python-graphviz


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    atk-1.0-2.38.0             |       h04ea711_2         348 KB  conda-forge
    ca-certificates-2024.6.2   |       hbcca054_0         152 KB  conda-forge
    cairo-1.18.0               |       h3faef2a_0         959 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    expat-2.6.2                |       h59595ed_0         134 KB  conda-forge
    font-ttf-dejavu-sans-mono-2.37|       hab24e00_0         388 KB  conda-forge
    font-ttf-inconsolata-3.000 |       h77eed37_0  

In [16]:
from graphviz import Digraph
from typing import Set

def create_graph(tree):
    dot = Digraph()
    
    for index, node in tree.all_nodes.items():
        dot.node(str(index), label=str(index))
        
    for index, node in tree.all_nodes.items():
        for child_index in node.children:
            dot.edge(str(index), str(child_index))
            
    return dot

graph = create_graph(tree)
graph.attr(layout = 'dot')
graph.render('tree_graph',format='png', cleanup = True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'tree_graph.png'